In [1]:
import numpy as np
from scipy.stats import t
from scipy.stats import norm
from math import exp
import pandas as pd
import statsmodels.api as sm

## 1. This exercise analyzes daily data for MSFT from January 2, 2012, to December 31, 2024. Using the adjusted closing prices, compute the daily log returns. Although some autocorrelation and volatility clustering may be present, assume that the returns are i.i.d. For bootstrapping, set $B = 1000$ and use a random seed of 432 (i.e., np.random.seed(432)inPython). TheinitialinvestmentisW0 =105.

In [2]:
import yfinance as yf

In [3]:
# Microsoft daily adjusted closing prices
msft = yf.download('MSFT', start = '2012-01-02', end = '2024-12-31', interval = '1d',
                  auto_adjust = False)['Adj Close']

# Convert to log returns
log_returns = np.log(msft / msft.shift(1))
log_returns = log_returns.dropna().reset_index()
log_returns.describe()

[*********************100%***********************]  1 of 1 completed


Ticker,Date,MSFT
count,3268,3268.000000
mean,2018-07-02 16:39:48.249694208,0.000918
min,2012-01-04 00:00:00,-0.159453
25%,2015-04-06 18:00:00,-0.006706
50%,2018-07-02 12:00:00,0.000728
75%,2021-09-29 06:00:00,0.009340
max,2024-12-30 00:00:00,0.132929
std,NaN,0.016364


In [4]:
# set random seed
np.random.seed(432)

# For bootstrapping
B = 1000

# Initial Investment
W_0 = 10e+5

### (a) Compute nonparametric estimates of VaR$_{0.01}$ and ES$_{0.01}$.

In [5]:
# Sort the returns
sorted_returns = log_returns.sort_values('MSFT').reset_index()

# product of n and alpha
index = int(.01*len(log_returns))

# get r_na
r_na = sorted_returns['MSFT'].iloc[index]

# Calculate non parametric VaR
npVaR_01 = W_0 * (np.exp(r_na) - 1)
print('The nonparametric estimation of VaR_0.01 =', np.round(npVaR_01, 3))

The nonparametric estimation of VaR_0.01 = -42095.949


In [6]:
# Profit
X_t = W_0 * (np.exp(log_returns['MSFT']) -1)

# calculate nonparametric ES
npES = np.mean(X_t * (X_t <= npVaR_01)) / 0.01

print('The nonparametic estimatino of ES =', np.round(npES, 3))

The nonparametic estimatino of ES = -59888.657


### (b) Compute parametric estimates of VaR$_{0.01}$ and ES$_{0.01}$ assuming that the returns are normally distributed.

In [7]:
# Parametric normal VaR
mu_hat, sigma_hat = norm.fit(log_returns['MSFT'])
norm_VaR_01 = W_0 * (np.exp(mu_hat + sigma_hat * norm.ppf(0.01)) - 1)

print('The parametric estimation of VaR_0.01 assuming r_t is normally distributed is', 
      np.round(norm_VaR_01, 3))

The parametric estimation of VaR_0.01 assuming r_t is normally distributed is -36462.196


In [8]:
# Parametric ES
norm_ES = np.mean(X_t * (X_t <= norm_VaR_01)) / 0.01

print('The corresponding parametric estimation of ES =', np.round(norm_ES, 3))

The corresponding parametric estimation of ES = -92046.504


### (c) Compute parametric estimates of VaR$_{0.01}$ and ES$_{0.01}$ assuming that assuming that the returns are $t$-distributed.

In [9]:
# fit t-distribution to data
v_hat, mu_hat, sigma_hat = t.fit(log_returns['MSFT'])

# parametric t-distributed VaR
t_VaR_01 = W_0 * (np.exp(mu_hat + sigma_hat * t.ppf(0.01, df=v_hat)) - 1)

print('The parametric estimation of VaR_0.01 assuming r_t is t-distributed is', np.round(t_VaR_01, 3))

The parametric estimation of VaR_0.01 assuming r_t is t-distributed is -42949.819


In [10]:
# Parametric ES
t_ES = np.mean(X_t * (X_t <= t_VaR_01)) / 0.01

print('The corresponding parametric estimation of ES =', np.round(t_ES, 3))

The corresponding parametric estimation of ES = -56007.65


### (d) Compare the estimates in (a), (b), and (c). Which do you feel are most realistic?

The nonparametric VaR estimate is likely the most realistic, since it avoids strong distributional assumptions and reflects the empirical distribution of returns. The normal distribution estimates the least tail risk, while the t-distribution estimates the most tail risk. The nonparametric method, while sample-dependent, offers the most direct and assumption-free estimate of risk.

### (e) Construct 95% symmetric bootstrap confidence intervals for VaR$_{0.01}$ and ES$_{0.01}$ using nonparametric estimators.

In [11]:
# Bootstrapping
boot_samples = np.zeros((B, 2))
n = len(log_returns)

for i in range(B):
    
    # Create bootstrap sample w/ replacement
    sample_returns = np.random.choice(log_returns['MSFT'], replace = True, size=n)
    sorted_returns = np.sort(sample_returns) # sort returns
    rb_na = sorted_returns[index] # alpha sample quantile
    Xb_t = W_0 * (np.exp(sample_returns) - 1)
    
    # estimate VaR
    VaR_est = W_0 * (np.exp(rb_na) - 1)

    # Estimate ES
    ES_est = np.mean(Xb_t * (Xb_t <= VaR_est)) / 0.01
    
    # list of estimated VaR's
    boot_samples[i] = [VaR_est, ES_est]

In [12]:
# Theta distribution of VaR
theta_dist = boot_samples[:, 0] - npVaR_01

# 95th quantile
q_95 = np.percentile(theta_dist, 95)

# Symmetric 95% confidence interval
VaR_ci = [npVaR_01 - q_95, npVaR_01 + q_95]
print('The 95% symmetric bootstrap confidence interval for the nonparametric estimation of VaR_01 is', 
      np.round(VaR_ci, 3))

The 95% symmetric bootstrap confidence interval for the nonparametric estimation of VaR_01 is [-44497.208 -39694.691]


In [13]:
# Theta distribution of ES
theta_dist = boot_samples[:, 1] - npES

# 95th percentile
q_95 = np.percentile(theta_dist, 95)

# Symmetric 95% confidence interval
ES_ci = [npES - q_95, npES + q_95]
print('The 95% symmetric bootstrap confidence interval for the nonparametric estimation of ES_01 is', 
      np.round(ES_ci, 3))

The 95% symmetric bootstrap confidence interval for the nonparametric estimation of ES_01 is [-66988.099 -52789.214]


### (f) Estimate the left tail index via regression (set m = 0:1n). Using $\alpha_0 = 0.01$, compute VaR$_{0.001}$ and ES$_{0.001}$.

In [14]:
# Calculate the sample size
n = len(log_returns)
# Set the significance level alpha_0
alpha_0 = 0.1
# Calculate the number of extreme values to use (m = alpha_0 * n)
m = int(n * alpha_0)

# Create an array of indices from 1 to m
i_vals = np.arange(1, m+1)

# Calculate log(i/n) which will be the dependent variable in the regression
log = np.log(i_vals / n)

# Sort the returns in ascending order
X_t = np.sort(X_t)

# Transform the extreme values using negative log of negative values
Z_i = -np.log(-X_t[i_vals])

# Add a constant term to the independent variables for the regression intercept
X = sm.add_constant(Z_i)

# Fit an Ordinary Least Squares regression model
model = sm.OLS(log, X).fit()

# Display the regression results summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.969
Model:                            OLS   Adj. R-squared:                  0.969
Method:                 Least Squares   F-statistic:                 1.011e+04
Date:                Mon, 22 Sep 2025   Prob (F-statistic):          2.19e-246
Time:                        22:02:21   Log-Likelihood:                 114.73
No. Observations:                 326   AIC:                            -225.5
Df Residuals:                     324   BIC:                            -217.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         23.9437      0.271     88.357      0.0

In [15]:
kappa = model.params[1] # Extract the kappa parameter (second parameter) from the model

In [16]:
# Calculate Value at Risk (VaR) at alpha_0 confidence level
VaR_alpha0 = X_t[int(n * alpha_0)]

# Set target confidence level alpha for risk calculation
alpha = 0.001

# Calculate VaR at target confidence level using Pareto extrapolation
VaR_alpha = VaR_alpha0 * (alpha_0 / alpha) ** (1 / kappa)

print('VaR_0.001 =', VaR_alpha)

# Calculate Expected Shortfall (ES) at target confidence level
# For Pareto distribution, ES_alpha = (kappa/(kappa-1)) * VaR_alpha
ES_alpha = (kappa / (kappa - 1)) * VaR_alpha

print('ES_0.001 =', ES_alpha)

VaR_0.001 = -91037.64383230629
ES_0.001 = -145283.54140661375
